In [13]:
# import pandas
import pandas as pd  

# from apyori import apriori
from apyori import apriori

In [14]:
# load data 
movie_data = pd.read_csv('https://raw.githubusercontent.com/pirandello/apriori/master/movie_dataset.csv', 
                         header = None)

* explore the dataset

In [15]:
movie_data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,The Revenant,13 Hours,Allied,Zootopia,Jigsaw,Achorman,Grinch,Fast and Furious,Ghostbusters,Wolverine,Mad Max,John Wick,La La Land,The Good Dunosaur,Ninja Turtles,The Good Dunosaur Bad Moms,2 Guns,Inside Out,Valerian,Spiderman 3
1,Beirut,Martian,Get Out,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Deadpool,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,X-Men,Allied,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Ninja Turtles,Moana,Ghost in the Shell,Ralph Breaks the Internet,John Wick,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
movie_data.shape

(7501, 20)

* transform dataframe to list of lists (suitable format for apyori)

In [20]:
# preprocess
records = []
for i in range(0, movie_data.shape[0]):
    records.append([str(movie_data.values[i,j]) for j in range(0, 20)])

* instantiate apriori and set the input params based on following constraints:
    * we want only movies that are purchased at least 40 times
    * the minimum confidence for the rules is 20%
    * the minumum lift is 3

In [21]:
association_rules = apriori(records, min_support = 40/7501, min_condifidence = 0.20, min_lift = 3)
association_results = list(association_rules)

* how many association rules did we obtained ?

In [22]:
print(len(association_results))
# 34 rules

34


* print the first association rule

In [23]:
print(association_results[0])

RelationRecord(items=frozenset({'Red Sparrow', 'Green Lantern'}), support=0.005732568990801226, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Green Lantern'}), items_add=frozenset({'Red Sparrow'}), confidence=0.0722689075630252, lift=3.7908326967150496), OrderedStatistic(items_base=frozenset({'Red Sparrow'}), items_add=frozenset({'Green Lantern'}), confidence=0.3006993006993007, lift=3.790832696715049)])


* convert association rules to DataFrame 
    * use these columns: title_1, title_2, support, confidence, lift

In [24]:
results = []
for item in association_results:

    # first index of inner list
    # contains base item and add item
    pair = item[0]
    items = [x for x in pair]

    value0 = str(items[0])
    value1 = str(items[1])

    # second index of inner list
    value2  = str(item[1])[:7]

    #third index of the list located at the 0th of the thrid index of the inner list
    value3 = str(item[2][0][2])[:7]
    value4 = str(item[2][0][3])[:7]

    rows = (value0, value1, value2, value3, value4)
    results.append(rows)

labels = ['Title 1', 'Title 2', 'Support', 'Confidence', 'Lift']
movie_suggestion = pd.DataFrame.from_records(results, columns = labels)

print(movie_suggestion)

          Title 1                Title 2  Support Confidence     Lift
0     Red Sparrow          Green Lantern  0.00573    0.07226  3.79083
1       Star Wars          Green Lantern  0.00586    0.07394  4.70081
2   Kung Fu Panda                Jumanji  0.01599    0.16282  3.29199
3         Jumanji           Wonder Woman  0.00533    0.05427  3.84065
4     Spiderman 3  The Spy Who Dumped Me  0.00799    0.12145  4.12241
5          Intern           The Revenant  0.00533    0.05594  3.10841
6     Red Sparrow                    nan  0.00573    0.07226  3.79083
7             nan              Star Wars  0.00586    0.07394  4.70081
8      Tomb Rider                 Intern  0.00866    0.08819  3.16532
9   Ninja Turtles                 Intern  0.00719    0.07552  3.20061
10     Tomb Rider            Spiderman 3  0.00573    0.08704  3.12402
11     Tomb Rider                 Intern  0.00599    0.08395  3.01314
12     Tomb Rider            World War Z  0.00666    0.06993  3.34105
13     Tomb Rider   

Support(x) = Number of times x appears/total number of items

Confidence(x-> y) = Transactions containing both x and y / Transactions containing x

Lift(x -> y) = Confidence(x -> y) / Support(y)